# Introduction

Do higher film budgets lead to more box office revenue? Let's find out if there's a relationship using the movie budgets and financial performance data that I've scraped from [the-numbers.com](https://www.the-numbers.com/movie/budgets) on **May 1st, 2018**. 

<img src=https://i.imgur.com/kq7hrEh.png>

# Import Statements

In [3]:
import pandas as pd
import matplotlib.pyplot as plt


# Notebook Presentation

In [4]:
pd.options.display.float_format = '{:,.2f}'.format

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Read the Data

In [5]:
data = pd.read_csv('cost_revenue_dirty.csv')
data.head()
data.info()
data.sample()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5391 entries, 0 to 5390
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Rank                   5391 non-null   int64 
 1   Release_Date           5391 non-null   object
 2   Movie_Title            5391 non-null   object
 3   USD_Production_Budget  5391 non-null   object
 4   USD_Worldwide_Gross    5391 non-null   object
 5   USD_Domestic_Gross     5391 non-null   object
dtypes: int64(1), object(5)
memory usage: 252.8+ KB


,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
846,956,2/8/1996,Chain Reaction,"$55,000,000","$60,209,334","$21,226,204"


# Explore and Clean the Data

**Challenge**: Answer these questions about the dataset:
1. How many rows and columns does the dataset contain?
2. Are there any NaN values present?
3. Are there any duplicate rows?
4. What are the data types of the columns?

In [6]:
total_rows = data.shape[0]
total_cols = data.shape[1]
print(f'Total rows: {total_rows}')
print(f'Total columns: {total_cols}')



has_nan_values = data.isna().values.any()
has_any_duplicates = data.duplicated().values.any()
print(f"Total Duplicates: {data.duplicated().sum()}")

print(f'Has NaN values: {has_nan_values}')
print(f"Data Types: {data.dtypes}")
print(f'Has any duplicates: {has_any_duplicates}')

data.describe()

Total rows: 5391
Total columns: 6
Total Duplicates: 0
Has NaN values: False
Data Types: Rank                      int64
Release_Date             object
Movie_Title              object
USD_Production_Budget    object
USD_Worldwide_Gross      object
USD_Domestic_Gross       object
dtype: object
Has any duplicates: False


,Rank
count,"5,391.00"
mean,"2,696.00"
std,"1,556.39"
min,1.00
25%,"1,348.50"
50%,"2,696.00"
75%,"4,043.50"
max,"5,391.00"


### Data Type Conversions

**Challenge**: Convert the `USD_Production_Budget`, `USD_Worldwide_Gross`, and `USD_Domestic_Gross` columns to a numeric format by removing `$` signs and `,`. 
<br>
<br>
Note that *domestic* in this context refers to the United States.

In [7]:
data["USD_Production_Budget"] = data["USD_Production_Budget"].str.replace(',', '').str.replace('$', '').astype('float')
data["USD_Worldwide_Gross"] = data["USD_Worldwide_Gross"].str.replace(',', '').str.replace('$', '').astype('float')
data["USD_Domestic_Gross"] = data["USD_Domestic_Gross"].str.replace(',', '').str.replace('$', '').astype('float')

**Challenge**: Convert the `Release_Date` column to a Pandas Datetime type. 

In [21]:
print(f"Data Types: {data.dtypes}")

Data Types: Rank                       int64
Release_Date              object
Movie_Title               object
USD_Production_Budget    float64
USD_Worldwide_Gross      float64
USD_Domestic_Gross       float64
dtype: object


In [8]:
data["Release_Date"] = pd.to_datetime(data["Release_Date"])

### Descriptive Statistics

**Challenge**: 

1. What is the average production budget of the films in the data set?
2. What is the average worldwide gross revenue of films?
3. What were the minimums for worldwide and domestic revenue?
4. Are the bottom 25% of films actually profitable or do they lose money?
5. What are the highest production budget and highest worldwide gross revenue of any film?
6. How much revenue did the lowest and highest budget films make?

In [9]:
avg_budget = data["USD_Production_Budget"].mean()
avg_worldwide_gross = data["USD_Worldwide_Gross"].mean()
min_domestic_revenue = data["USD_Domestic_Gross"].sum()
min_worldwide_revenue = data["USD_Worldwide_Gross"].sum()

data.describe(include = 'all')
print(data["Movie_Title"].value_counts())

Movie_Title
Ben-Hur                             3
King Kong                           3
Home                                3
The Birth of a Nation               2
The Gambler                         2
                                   ..
Insomnia                            1
Spirit: Stallion of the Cimarron    1
The Importance of Being Earnest     1
The Believer                        1
Story of Bonnie and Clyde, The      1
Name: count, Length: 5318, dtype: int64


In [28]:
bottom25_threshold = data["USD_Production_Budget"].quantile(0.25)
bottom25 = data[data["USD_Production_Budget"] <= bottom25_threshold]
#print(bottom25)

#data.info()

profitable = [index for index, series in bottom25.iterrows() if series['USD_Production_Budget'] > (series['USD_Worldwide_Gross'] + series['USD_Domestic_Gross'])]

#print(f"Profit/Loss: {bottom25['USD_Production_Budget'].sum() - (bottom25['USD_Worldwide_Gross'].sum()+data['USD_Domestic_Gross'].sum())}")
print(profitable)

[1, 6, 8, 15, 17, 27, 50, 66, 71, 75, 78, 120, 138, 168, 177, 198, 253, 274, 348, 351, 401, 405, 429, 439, 468, 474, 500, 502, 555, 585, 588, 635, 646, 660, 661, 699, 733, 839, 850, 892, 900, 945, 969, 979, 1044, 1045, 1049, 1052, 1059, 1075, 1076, 1080, 1090, 1112, 1113, 1141, 1145, 1211, 1219, 1223, 1228, 1231, 1254, 1272, 1279, 1284, 1306, 1335, 1371, 1384, 1412, 1417, 1430, 1431, 1437, 1471, 1494, 1501, 1511, 1521, 1560, 1579, 1602, 1655, 1666, 1667, 1675, 1684, 1692, 1745, 1751, 1773, 1775, 1789, 1819, 1866, 1916, 1917, 1938, 1953, 1954, 1971, 1984, 1991, 1995, 1999, 2028, 2042, 2043, 2052, 2063, 2069, 2081, 2097, 2113, 2117, 2121, 2147, 2155, 2156, 2157, 2189, 2211, 2228, 2244, 2254, 2296, 2301, 2309, 2310, 2360, 2365, 2371, 2374, 2375, 2393, 2396, 2405, 2408, 2409, 2410, 2412, 2439, 2443, 2452, 2457, 2462, 2465, 2492, 2498, 2503, 2512, 2513, 2514, 2520, 2521, 2528, 2539, 2546, 2579, 2580, 2581, 2585, 2587, 2601, 2602, 2611, 2625, 2630, 2631, 2641, 2656, 2663, 2675, 2688, 2696, 2

# Investigating the Zero Revenue Films

**Challenge** How many films grossed $0 domestically (i.e., in the United States)? What were the highest budget films that grossed nothing?

In [33]:
zero_rev_movies_in_us = data[data["USD_Domestic_Gross"] == 0].sort_values("USD_Production_Budget", ascending=False)
print(zero_rev_movies_in_us)

      Rank Release_Date                         Movie_Title  \
5388    96   2020-12-31                         Singularity   
5387   126   2018-12-18                             Aquaman   
5384   321   2018-09-03                   A Wrinkle in Time   
5385   366   2018-10-08                      Amusement Park   
5090   556   2015-12-31  Don Gato, el inicio de la pandilla   
...    ...          ...                                 ...   
4787  5371   2014-12-31                Stories of Our Lives   
3056  5374   2007-12-31                         Tin Can Man   
4907  5381   2015-05-19                    Family Motocross   
5006  5389   2015-09-29             Signed Sealed Delivered   
5007  5390   2015-09-29                A Plague So Pleasant   

      USD_Production_Budget  USD_Worldwide_Gross  USD_Domestic_Gross  
5388         175,000,000.00                 0.00                0.00  
5387         160,000,000.00                 0.00                0.00  
5384         103,000,000.00   

**Challenge**: How many films grossed $0 worldwide? What are the highest budget films that had no revenue internationally?

In [42]:
max_budget = data["USD_Production_Budget"].max()
zero_rev_movies_in_ww = data[(data["USD_Worldwide_Gross"] == 0) & (data["USD_Production_Budget"] == max_budget)]
print(zero_rev_movies_in_ww)

Empty DataFrame
Columns: [Rank, Release_Date, Movie_Title, USD_Production_Budget, USD_Worldwide_Gross, USD_Domestic_Gross]
Index: []


### Filtering on Multiple Conditions

In [44]:
subset = data.query('USD_Production_Budget < 1000000000 and USD_Worldwide_Gross > 0')
print(subset)

      Rank Release_Date                       Movie_Title  \
0     5293   1915-08-02             The Birth of a Nation   
2     5230   1916-12-24      20,000 Leagues Under the Sea   
3     5299   1920-09-17    Over the Hill to the Poorhouse   
4     5222   1925-01-01                    The Big Parade   
5     4250   1925-12-30                           Ben-Hur   
...    ...          ...                               ...   
5378   914   2017-10-02               Fifty Shades Darker   
5379  1295   2017-10-02            John Wick: Chapter Two   
5380    70   2017-10-03                Kong: Skull Island   
5381    94   2017-12-05  King Arthur: Legend of the Sword   
5382  1254   2017-12-05                          Snatched   

      USD_Production_Budget  USD_Worldwide_Gross  USD_Domestic_Gross  
0                110,000.00        11,000,000.00       10,000,000.00  
2                200,000.00         8,000,000.00        8,000,000.00  
3                100,000.00         3,000,000.00      

**Challenge**: Use the [`.query()` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) to accomplish the same thing. Create a subset for international releases that had some worldwide gross revenue, but made zero revenue in the United States. 

Hint: This time you'll have to use the `and` keyword.

In [45]:
subset1 = data.query('USD_Domestic_Gross  == 0 and USD_Worldwide_Gross > 0')
print(subset1)

      Rank Release_Date              Movie_Title  USD_Production_Budget  \
71    4310   1956-02-16                 Carousel           3,380,000.00   
1579  5087   2001-02-11  Everything Put Together             500,000.00   
1744  3695   2001-12-31                 The Hole           7,500,000.00   
2155  4236   2003-12-31                  Nothing           4,000,000.00   
2203  2513   2004-03-31                The Touch          20,000,000.00   
...    ...          ...                      ...                    ...   
5340  1506   2017-04-14      Queen of the Desert          36,000,000.00   
5348  2225   2017-05-05        Chāi dàn zhuānjiā          23,000,000.00   
5360  4832   2017-07-03                Departure           1,100,000.00   
5372  1856   2017-08-25                Ballerina          30,000,000.00   
5374  4237   2017-08-25     Polina danser sa vie           4,000,000.00   

      USD_Worldwide_Gross  USD_Domestic_Gross  
71               3,220.00                0.00  
157

### Unreleased Films

**Challenge**:
* Identify which films were not released yet as of the time of data collection (May 1st, 2018).
* How many films are included in the dataset that have not yet had a chance to be screened in the box office? 
* Create another DataFrame called data_clean that does not include these films. 

In [ ]:
# Date of Data Collection
scrape_date = pd.Timestamp('2018-5-1')

In [46]:
data.sample(5)

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
4661,2842,2014-08-22,When the Game Stands Tall,"15,000,000.00","30,138,912.00","30,127,963.00"
3977,3171,2011-09-12,Young Adult,"12,000,000.00","22,750,356.00","16,311,571.00"
256,1691,1979-08-15,Apocalypse Now,"31,500,000.00","81,250,488.00","78,800,000.00"
1173,1884,1998-11-20,The Rugrats Movie,"28,000,000.00","140,894,685.00","100,494,685.00"
4839,5260,2015-02-27,Butterfly Girl,"180,000.00","17,503.00","17,503.00"


### Films that Lost Money

**Challenge**: 
What is the percentage of films where the production costs exceeded the worldwide gross revenue? 

# Seaborn for Data Viz: Bubble Charts

### Plotting Movie Releases over Time

**Challenge**: Try to create the following Bubble Chart:

<img src=https://i.imgur.com/8fUn9T6.png>



# Converting Years to Decades Trick

**Challenge**: Create a column in `data_clean` that has the decade of the release. 

<img src=https://i.imgur.com/0VEfagw.png width=650> 

Here's how: 
1. Create a [`DatetimeIndex` object](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html) from the Release_Date column. 
2. Grab all the years from the `DatetimeIndex` object using the `.year` property.
<img src=https://i.imgur.com/5m06Ach.png width=650>
3. Use floor division `//` to convert the year data to the decades of the films.
4. Add the decades as a `Decade` column to the `data_clean` DataFrame.

### Separate the "old" (before 1969) and "New" (1970s onwards) Films

**Challenge**: Create two new DataFrames: `old_films` and `new_films`
* `old_films` should include all the films before 1969 (up to and including 1969)
* `new_films` should include all the films from 1970 onwards
* How many films were released prior to 1970?
* What was the most expensive film made prior to 1970?

# Seaborn Regression Plots

**Challenge**: Use Seaborn's `.regplot()` to show the scatter plot and linear regression line against the `new_films`. 
<br>
<br>
Style the chart

* Put the chart on a `'darkgrid'`.
* Set limits on the axes so that they don't show negative values.
* Label the axes on the plot "Revenue in \$ billions" and "Budget in \$ millions".
* Provide HEX colour codes for the plot and the regression line. Make the dots dark blue (#2f4b7c) and the line orange (#ff7c43).

Interpret the chart

* Do our data points for the new films align better or worse with the linear regression than for our older films?
* Roughly how much would a film with a budget of $150 million make according to the regression line?

# Run Your Own Regression with scikit-learn

$$ REV \hat ENUE = \theta _0 + \theta _1 BUDGET$$

**Challenge**: Run a linear regression for the `old_films`. Calculate the intercept, slope and r-squared. How much of the variance in movie revenue does the linear model explain in this case?

# Use Your Model to Make a Prediction

We just estimated the slope and intercept! Remember that our Linear Model has the following form:

$$ REV \hat ENUE = \theta _0 + \theta _1 BUDGET$$

**Challenge**:  How much global revenue does our model estimate for a film with a budget of $350 million? 